# Investigate dynamic services

## Setup

In [ ]:
%%capture
!{sys.executable} -m pip install httpx pydantic
!{sys.executable} -m pip install pandas

In [ ]:
from uuid import UUID

import os
import httpx
from pydantic import validate_arguments


def check():
    r = httpx.get("http://" + os.environ.get('PREFIX_STACK_NAME') + "_director:8080/v0/")
    return r


@validate_arguments
def get_service(service_id: str):
    r = httpx.get(
        "http://" + os.environ.get('PREFIX_STACK_NAME') + f"_director:8080/v0/running_interactive_services/{service_id}"
    )
    return r


@validate_arguments
def list_services(user_id: int, project_id: UUID):
    r = httpx.get(
        "http://" + os.environ.get('PREFIX_STACK_NAME') + f"_director:8080/v0/running_interactive_services",
        params={"user_id": user_id, "project_id": project_id},
    )
    return r


@validate_arguments
def delete_service(service_id: str, save_state: bool):
    r = httpx.delete(
        "http://" + os.environ.get('PREFIX_STACK_NAME') + "_director:8080/v0/running_interactive_services/{service_id}",
        params={"save_state": save_state},
    )
    return r

In [ ]:
import pandas as pd


def to_datafield(r: httpx.Response):
    df = pd.json_normalize(r.json()["data"])
    return df

In [ ]:
import httpx

r = check()
r.url, r.status_code, r.text

# Investigate *running* dynamic services



## For a user & project, list all the dynamic services

In [ ]:
user_id = 72952
project_id = "67406d1c-31af-11ec-8033-02420a0b2de3"


r = list_services(user_id, project_id)
r.url, r.status_code

In [ ]:
df = to_datafield(r)
df.info()

In [ ]:
df[["service_uuid", "service_key", "service_version", "service_state"]]

## Given a service, what is the project it belongs to and who is the owner?


In [ ]:
service_id = "88fabeb4-0a67-4514-a81d-6dff688b60bf"

r = get_service(service_id)
d = r.json()["data"]
d

In [ ]:
r = list_services(user_id=d["user_id"], project_id=d["project_id"])
df = to_datafield(r)
df[["service_uuid", "service_key", "service_version", "service_state"]]